In [1]:
from weiner_variation.sim.data_structures import DrawDurations
from weiner_variation.sim.process import PASS_SEQUENCE, DIAMETER, TEMPERATURE, create_in_profile, REVERSING_PAUSE_DURATION, DIAMETER_STD, TEMPERATURE_STD
from weiner_variation.sim.config import SAMPLE_COUNT, FIELDS, SEED
from weiner_variation.config import DATA_DIR

import pandas as pd
import numpy as np
from scipy.stats import norm, weibull_min
from copy import deepcopy
import tqdm
from multiprocessing import Pool

In [2]:
import pyroll.basic as pr
#import pyroll.jmak_recrystallization

In [3]:
DIAMETER_STD = DIAMETER_STD
TEMPERATURE_STD = TEMPERATURE_STD
OUTPUT_FILENAME = "sim_durations_results.csv"

In [4]:
df_durations = pd.read_csv(DATA_DIR / "duo_pauses_dist.csv", header=0, index_col=0)
df_durations

,mean,median,std,min,max,shape,scale,fit_mean,fit_std
R1-R2,6.633088,6.4000,1.471388,5.115,15.955,8.212077,6.543489,6.170061,0.893318
R2-R3,4.435441,4.2500,0.995052,3.040,11.100,7.705520,4.407880,4.143458,0.636670
R3-R4,5.988015,5.6100,2.192780,4.015,21.000,7.029720,5.623389,5.261487,0.880575
R4-R5,5.654853,5.6625,0.470526,4.745,7.115,13.799551,5.710760,5.499720,0.487191
R5-R6,5.044559,5.0200,0.566902,3.635,6.030,8.707140,5.276168,4.988786,0.683739
R6-R7,5.195588,5.0850,0.588969,4.235,8.095,16.114621,5.085251,4.921435,0.375704
R7-R8,6.435882,6.1100,1.557230,4.705,13.170,8.146553,6.161134,5.807295,0.847121
R8-R9,6.080735,5.8800,0.723092,4.745,8.260,14.631003,5.900644,5.693526,0.476881
R9-R10,7.008235,7.0050,0.775194,5.260,9.215,11.127373,7.139707,6.822094,0.741851
R10-F1,9.233309,9.1775,0.399014,8.290,10.385,37.735730,9.203396,9.068859,0.302529


In [5]:
diameter_dist = norm(loc=DIAMETER, scale=DIAMETER_STD)
temperature_dist = norm(loc=TEMPERATURE, scale=TEMPERATURE_STD)
durations_dists = [weibull_min(c=r["shape"], scale=r["scale"]) for i, r in df_durations.iterrows()]

In [6]:
RNG = np.random.default_rng(SEED)
diameters = diameter_dist.rvs(random_state=RNG, size=SAMPLE_COUNT)
temperatures = temperature_dist.rvs(random_state=RNG, size=SAMPLE_COUNT)
durations = np.concatenate([
    d.rvs(random_state=RNG, size=(SAMPLE_COUNT, 1))
    for d in durations_dists[:-1]
], axis=1)

draws = [
    DrawDurations(d, t, dur)
    for d, t, dur in zip(diameters, temperatures, durations)
]

In [7]:
def worker(draw: DrawDurations):    
    ip = create_in_profile(
            diameter=draw.diameter,
            temperature=draw.temperature,
    )

    sequence = deepcopy(PASS_SEQUENCE)  
    
    transports = [u for u in sequence if isinstance(u, pr.Transport)]
    for t, d in zip(transports, draw.durations):
        t.duration = d
    
    sequence.solve(ip)       

    return (
        draw.__dict__.items() |> map$(t -> (("draw", t[0]), t[1])) |> dict
    ) | (
        FIELDS.items()
        |> starmap$((key, extractor) -> (
                sequence.units 
                |> filter$(u -> isinstance(u, pr.RollPass)) 
                |> map$(u -> ((key, u.label), extractor(u))) 
                |> filter$(t -> t[1] is not None)
            )
        )
        |> flatten
        |> dict            
    )

In [8]:
results = tqdm.tqdm(Pool().imap(worker, draws), total=SAMPLE_COUNT) |> list

  0%|                                                                                                                                                                                                                                                                                                                                                                                                                                                      | 0/1000 [00:00<?, ?it/s]Solution iteration of RollPass 'R8' exceeded the maximum iteration count of 100. Continuing anyway.
Solution iteration of RollPass 'R8' exceeded the maximum iteration count of 100. Continuing anyway.
  0%|▍                                                                                                                                                                                                                                                                                                                                      

RemoteTraceback: 
"""
Traceback (most recent call last):
  File "/home/maxwe/PycharmProjects/publication-weiner-variation/.venv_sim/lib/python3.11/site-packages/pyroll/core/unit/unit.py", line 152, in _solve_subunits
    last_profile = u.solve(last_profile)
                   ^^^^^^^^^^^^^^^^^^^^^
  File "/home/maxwe/PycharmProjects/publication-weiner-variation/.venv_sim/lib/python3.11/site-packages/pyroll/core/unit/unit.py", line 171, in solve
    self.reevaluate_cache()
  File "/home/maxwe/PycharmProjects/publication-weiner-variation/.venv_sim/lib/python3.11/site-packages/pyroll/core/hooks.py", line 380, in reevaluate_cache
    self.__cache__[n] = hook.get_result(self)
                        ^^^^^^^^^^^^^^^^^^^^^
  File "/home/maxwe/PycharmProjects/publication-weiner-variation/.venv_sim/lib/python3.11/site-packages/pyroll/core/hooks.py", line 277, in get_result
    result = f(instance)
             ^^^^^^^^^^^
  File "/home/maxwe/PycharmProjects/publication-weiner-variation/.venv_si

Solution iteration of RollPass 'R8' exceeded the maximum iteration count of 100. Continuing anyway.
Solution iteration of RollPass 'R8' exceeded the maximum iteration count of 100. Continuing anyway.
Solution iteration of RollPass 'R8' exceeded the maximum iteration count of 100. Continuing anyway.
Solution iteration of RollPass 'R8' exceeded the maximum iteration count of 100. Continuing anyway.
Solution iteration of RollPass 'R8' exceeded the maximum iteration count of 100. Continuing anyway.
Solution iteration of RollPass 'R8' exceeded the maximum iteration count of 100. Continuing anyway.
Solution iteration of RollPass 'R8' exceeded the maximum iteration count of 100. Continuing anyway.
Solution iteration of RollPass 'R8' exceeded the maximum iteration count of 100. Continuing anyway.
Solution iteration of RollPass 'R8' exceeded the maximum iteration count of 100. Continuing anyway.
Solution iteration of RollPass 'R8' exceeded the maximum iteration count of 100. Continuing anyway.


In [9]:
df = results |> filter$(e -> "error" not in e) |> enumerate |> dict |> pd.DataFrame |> .T |> .infer_objects()
df

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'results' is not defined


In [ ]:
df.to_csv(DATA_DIR / OUTPUT_FILENAME)